# Sympy tests

In [54]:
from sympy import symbols, cos, sin, asin, acos, pi, sqrt, atan2, Abs, simplify, pprint, solveset
from sympy.matrices import Matrix, Transpose
import numpy as np

In [2]:
dtr = pi/180.0
rtd = 180.0/pi

In [3]:
# init_printing()
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')

# DH Parameters
s = {alpha0: 0,     a0: 0,      d1: 0.75,  q1: q1,           # 1
     alpha1: -pi/2, a1: 0.35,   d2: 0,     q2: -pi/2 + q2,   # 2
     alpha2: 0,     a2: 1.25,   d3: 0,     q3: q3,           # 3
     alpha3: -pi/2, a3: -0.054, d4: 1.5,   q4: q4,           # 4
     alpha4: pi/2,  a4: 0,      d5: 0,     q5: q5,           # 5
     alpha5: -pi/2, a5: 0,      d6: 0,     q6: q6,           # 6
     alpha6: 0,     a6: 0,      d7: 0.303, q7: 0}            # G

def rot_x(q):
    T = Matrix([[1, 0, 0, 0],
                [0, cos(q), -sin(q), 0],
                [0, sin(q), cos(q), 0],
                [0, 0, 0, 1]])
    return T

def trans_x(d):
    T = Matrix([[1, 0, 0, d],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T

def rot_z(q):
    T = Matrix([[cos(q), -sin(q), 0, 0],
                [sin(q), cos(q), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T


def trans_z(d):
    T = Matrix([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, d],
                [0, 0, 0, 1]])
    return T

def rot_y(q):
    T = Matrix([[cos(q),  0, sin(q), 0],
                [0,       1,      0, 0],
                [-sin(q), 0, cos(q), 0],
                [0,       0,      0, 1]])
    return T



def dh_trans(alpha0, a0, d1, q1):
    T = rot_x(alpha0) * trans_x(a0) * rot_z(q1) * trans_z(d1)
    return T

T0_1 = dh_trans(alpha0, a0, d1, q1)
T0_1 = T0_1.subs(s)

T1_2 = dh_trans(alpha1, a1, d2, q2)
T1_2 = T1_2.subs(s)

T2_3 = dh_trans(alpha2, a2, d3, q3)
T2_3 = T2_3.subs(s)

T3_4 = dh_trans(alpha3, a3, d4, q4)
T3_4 = T3_4.subs(s)

T4_5 = dh_trans(alpha4, a4, d5, q5)
T4_5 = T4_5.subs(s)

T5_6 = dh_trans(alpha5, a5, d6, q6)
T5_6 = T5_6.subs(s)

T6_G = dh_trans(alpha6, a6, d7, q7)
T6_G = T6_G.subs(s)



T0_2 = T0_1 * T1_2

T0_3 = T0_2 * T2_3

T0_4 = T0_3 * T3_4

T0_5 = T0_4 * T4_5

T0_6 = T0_5 * T5_6

T0_G = T0_6 * T6_G

R2_corr = rot_z(pi/2) * rot_x(pi/2)
R3_corr = R2_corr
R4_corr = rot_z(pi) * rot_y(-pi/2)
R5_corr = R2_corr
R6_corr = R4_corr
RG_corr = R4_corr

T0_2_corr = T0_2 * R2_corr

T0_3_corr = T0_3 * R3_corr

T0_4_corr = T0_4 * R4_corr

T0_5_corr = T0_5 * R5_corr

T0_6_corr = T0_6 * R6_corr

T0_G_corr = T0_G * RG_corr


# T1 = dh_trans(45*dtr, 5, 2, 30*dtr)
# T2 = rot_x(45*dtr) * trans_x(5) * rot_z(30*dtr) * trans_z(2)

# print('T0_1 =')
# pprint(T0_1)
# print('T1_2 =')
# pprint(T1_2)
# print('T0_2 =')
# pprint(T0_2)

print('T0_1 =')
# pprint(T0_1)
print('T0_1 =')
pprint(T0_1.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T1_2 =')
# pprint(T1_2)
print('T1_2 =')
pprint(T1_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_2 =')
# pprint(T0_2)
print('T0_2 =')
pprint(T0_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T3_4 =')
# pprint(T3_4)
print('T3_4 =')
pprint(T3_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4 =')
# pprint(T0_4)
print('T0_4 =')
pprint(T0_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T4_5 =')
# pprint(T4_5)
print('T4_5 =')
pprint(T4_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5 =')
# pprint(T0_5)
print('T0_5 =')
pprint(T0_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T5_6 =')
# pprint(T5_6)
print('T5_6 =')
pprint(T5_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6 =')
# pprint(T0_6)
print('T0_6 =')
pprint(T0_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T6_G =')
# pprint(T6_G)
print('T6_G =')
pprint(T6_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_G =')
# pprint(T0_G)
print('T0_G =')
pprint(T0_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))






print('T0_2_corr =')
# pprint(T0_2_corr)
print('T0_2_corr =')
pprint(T0_2_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_3_corr =')
# pprint(T0_3_corr)
print('T0_3_corr =')
pprint(T0_3_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4_corr =')
# pprint(T0_4_corr)
print('T0_4_corr =')
pprint(T0_4_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5_corr =')
# pprint(T0_5_corr)
print('T0_5_corr =')
pprint(T0_5_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6_corr =')
# pprint(T0_6_corr)
print('T0_6_corr =')
pprint(T0_6_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_G_corr =')
pprint(T0_G_corr)
# print('T0_G_corr_simp =')
# pprint(simplify(T0_G_corr))
print('T0_G_corr =')
pprint(T0_G_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))




# print('T1 =', T1.evalf(3))
# print('T2 =', T2)



T0_1 =
T0_1 =
⎡1.0   0    0    0  ⎤
⎢                   ⎥
⎢ 0   1.0   0    0  ⎥
⎢                   ⎥
⎢ 0    0   1.0  0.75⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T1_2 =
T1_2 =
⎡ 0   1.0   0   0.35⎤
⎢                   ⎥
⎢ 0    0   1.0   0  ⎥
⎢                   ⎥
⎢1.0   0    0    0  ⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T0_2 =
T0_2 =
⎡ 0   1.0   0   0.35⎤
⎢                   ⎥
⎢ 0    0   1.0   0  ⎥
⎢                   ⎥
⎢1.0   0    0   0.75⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T3_4 =
T3_4 =
⎡1.0   0     0   -0.054⎤
⎢                      ⎥
⎢ 0    0    1.0   1.5  ⎥
⎢                      ⎥
⎢ 0   -1.0   0     0   ⎥
⎢                      ⎥
⎣ 0    0     0    1.0  ⎦
T0_4 =
T0_4 =
⎡ 0    0    1.0  1.85 ⎤
⎢                     ⎥
⎢ 0   -1.0   0     0  ⎥
⎢                     ⎥
⎢1.0   0     0   1.946⎥
⎢                     ⎥
⎣ 0    0     0    1.0 ⎦
T4_5 =
T4_5 =
⎡1.0   0    0     0 ⎤
⎢                   ⎥
⎢ 0    0   -1.0   0 ⎥
⎢                   ⎥
⎢ 0   1.0   0     0 ⎥
⎢

In [78]:
# Test Forward Kinematics

# Var 1
# joint_1 = -0.1
# joint_2 = -0.18
# joint_3 = 0.64
# joint_4 = 3.91
# joint_5 = -1.15
# joint_6 = -5.48

# Var 2
joint_1 = -0.1
joint_2 = -0.18
joint_3 = 0.64
joint_4 = 3.91
joint_5 = -1.15
joint_6 = -5.48


print('T0_G =')
pprint(T0_G.evalf(15, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6}))


print('T0_G_corr =')
pprint(T0_G_corr.evalf(15, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6}))

O = T0_G.evalf(6, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6})


T0_G =
⎡-0.36296949297962  0.92158178802293   0.137623236230578    1.48079285371809 ⎤
⎢                                                                            ⎥
⎢0.754533232047088  0.20403127784435   0.623739400228795   0.0446021126954374⎥
⎢                                                                            ⎥
⎢0.546747426974003  0.330239679090296  -0.769421214583823   1.0323609926516  ⎥
⎢                                                                            ⎥
⎣        0                  0                  0                  1.0        ⎦
T0_G_corr =
⎡0.137623236230578   -0.92158178802293   -0.36296949297962   1.48079285371809 
⎢                                                                             
⎢0.623739400228795   -0.20403127784435   0.754533232047088  0.0446021126954374
⎢                                                                             
⎢-0.769421214583823  -0.330239679090296  0.546747426974003   1.0323609926516  
⎢                                

In [87]:
# Inverse Kinematics Pipeline
# O = Matrix([[0.137623,   -0.921582,  -0.36297,   1.48079],
#             [0.623739,   -0.204031,  0.754533,  0.0446021],
#             [-0.769421,  -0.33024,   0.546747,   1.03236],
#             [0,           0,         0,          1.0]])

# Position of EE
px = O[0,3] # 1.48079
py = O[1,3] # 0.0446021
pz = O[2,3] # 1.03236

# Orientation of EE
# roll = -0.543361935706796
# pitch = 1.02689551597329
# yaw = 1.35363377393308



def get_rpy(m):
    # Source: http://planning.cs.uiuc.edu/node103.html
    alpha = atan2(m[1,0], m[0,0])
    beta = atan2(-m[2,0], sqrt(m[2,1] + m[2,2]))
    gamma = atan2(m[2,1], m[2,2])
    return gamma, beta, alpha

def get_rot_mat(r, p, y):
    Rxyz =  rot_z(y) * rot_y(p) * rot_x(r)
    return Rxyz


roll, pitch, yaw = get_rpy(O)
print('roll =', roll)
print('pitch =', pitch)
print('yaw =', yaw)

# Get rotation matrix from roll/pitch/yaw
# R = get_rot_mat(roll, pitch, yaw)
R = O[0:3, 0:3]
print('R =')
pprint(R)

# Distance from wrist center to end-effector
d = s[d7]

# Wrist Center
xc = px - d * R[0, 2]
yc = py - d * R[1, 2]
zc = pz - d * R[2, 2]

print('xc =', xc)
print('yc =', yc)
print('zc =', zc)

# theta1
theta_1 = atan2(yc, xc)
print('theta_1 =', theta_1.evalf())

# joint_2 center
x0 = s[a1]
z0 = s[d1]
print('x0 =', x0, ', z0 =', z0)

# Link
l1 = s[a2]
print('s[a3] = ', s[a3], ', s[d4] =', s[d4])
l2 = sqrt(s[a3]**2 + s[d4]**2)
theta30 = atan2(Abs(s[a3]), s[d4])
print('l1 =', l1, ', l2 =', l2)
print('theta30 =', theta30)

# Dist from {0} to {wc}
dwc = sqrt((sqrt(xc**2+yc**2)-x0)**2 + (zc-z0)**2)
print('dwc =', dwc)

# Check validity
assert(l1 + l2 > dwc)

# Cosine law for cos(pi - beta):
D = (l1**2 + l2**2 - dwc**2) / (2 * l1 * l2)
# theta_3 = pi/2 - theta30 - atan2(sqrt(1 - D**2), D) # here is possible a second solution when elbow-down with "-" sign
# theta_3_alt = pi/2 - theta30 - atan2(-sqrt(1 - D**2), D) # alt solution
theta_3 = pi/2 - theta30 - acos(D) # here is possible a second solution when elbow-down with "-" sign
theta_3_alt = pi/2 - theta30 + acos(D) # alt solution
print('D =', D)
print('theta_3 =', theta_3.evalf())
# print('theta_3_alt =', theta_3_alt.evalf())

# angle to l2 from l1
bet = pi/2 + theta_3 + theta30
print('bet =', bet)

# Find angle to {0} - {wc} line
gam = atan2(sqrt(xc**2+yc**2) - x0, zc - z0)
print('gam =', gam)

# Find angle between {0} - {wc} /_ l1
# cos (alph) = E
E = (l1**2 + dwc**2 - l2**2) / (2 * l1 * dwc)
alph = acos(E)
print('E =', E)
print('alph =', alph)

# if 0 < beta < pi then theta_2 = gam - alph else theta_2 = gam + alph
if 0 < beta and beta < pi:
    theta_2 = gam - alph
else:
    theta_2 = gam + alph
print('theta_2 =', theta_2)

pitch2 = theta_2 + theta_3
print('pitch2 =', pitch2.evalf())

# Check by feeding angles to forward kinematics
t04 = T0_4.evalf(subs={q1: theta_1, q2: theta_2, q3: theta_3, q4: 0})
print('wc_calc =')
pprint(t04)

# Wrist center should be equal to what we've started from
print('xc_diff =', xc - t04[0,3])
print('yc_diff =', yc - t04[1,3])
print('zc_diff =', zc - t04[2,3])
assert(abs(xc - t04[0,3]) < 1e-6)
assert(abs(yc - t04[1,3]) < 1e-6)
assert(abs(zc - t04[2,3]) < 1e-6)

# Get R0_4 with theta_4 = 0
R0_4 = t04[0:3, 0:3]
print('R0_4 =')
pprint(R0_4)

R4_G = Transpose(R0_4) * R[0:3, 0:3]
print('R4_G =')
pprint(R4_G)
# print('rpy1 =', get_rpy(RG_4))

R4_G_sym = rot_z(q4) * rot_y(q5+pi) * rot_z(q6)
print('R4_G_sym =')
pprint(simplify(R4_G_sym))

R4_G_sym1 = rot_z(q4) * rot_x(pi/2) * rot_z(q5) * rot_x(-pi/2) * rot_z(q6)
print('R4_G_sym1 =')
pprint(simplify(R4_G_sym1))

theta_5 = acos(R4_G[2,2])
print('theta_5 =', theta_5)

theta_4 = atan2(R4_G[1,2], R4_G[0,2])
print('theta_6 =', theta_6)

theta_4 = atan2(-R4_G[2,1], R4_G[2,0])
print('theta_4 =', theta_4)

R_check = R4_G_sym1.evalf(subs = {q4: theta_4, q5: theta_5, q6: theta_6})
print('R_check =')
pprint(R_check)

T0_G_check = T0_G.evalf(subs = {q1: theta_1, q2: theta_2, q3: theta_3, q4: theta_4, q5: theta_5, q6: theta_6})
print('T0_G_check =')
pprint(T0_G_check)



# T3_6 = rot_z(q4) * T4_5 * T5_6
# print('T3_6 =')
# t36 = T3_6.evalf(6, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6})
# pprint(t36)
# print('rpy2 =', get_rpy(t36))


# sol = solveset(ex, q3)
# pprint(sol.evalf())




roll = -0.405427 + pi
pitch = 1.17234*I
yaw = -1.12242 + pi
R =
⎡-0.36297  0.921582  0.137623 ⎤
⎢                             ⎥
⎢0.754533  0.204031  0.623739 ⎥
⎢                             ⎥
⎣0.546747  0.33024   -0.769421⎦
xc = 1.43909291595221
yc = -0.144390915632248
zc = 1.26549566018581
theta_1 = -0.0999999998690740
x0 = 0.35 , z0 = 0.75
s[a3] =  -0.054 , s[d4] = 1.5
l1 = 1.25 , l2 = 1.50097168527591
theta30 = 0.0359844600820516
dwc = 1.21146605157014
D = 0.625665634848134
theta_3 = 0.640000059260550
bet = -0.894811807452295 + pi
gam = 1.13126662698718
E = 0.256625926281498
alph = 1.31126673438799
theta_2 = -0.180000107400813
pitch2 = 0.459999951859737
wc_calc =
⎡ 0.441730172683088   -0.0998334165165562   0.891575988622395    1.43909291595
⎢                                                                             
⎢-0.0443208519679908  -0.995004165291097   -0.0894559843397838  -0.14439091563
⎢                                                                             
⎢ 0.89605

In [6]:
# Test roll/pitch/yaw transforms

# Get roll/pitch/yaw angles
gamma, beta, alpha = get_rpy(O)

print('alpha =')
pprint(alpha)
print('beta =')
pprint(beta)
print('gamma =')
pprint(gamma)

# Reconstruct rotation matrix from roll/pitch/yaw

Rxyz = get_rot_mat(gamma, beta, alpha)
print('Rxyz =')
pprint(Rxyz)

gamma, beta, alpha = get_rpy(Rxyz)
print('alpha =')
pprint(alpha)
print('beta =')
pprint(beta)
print('gamma =')
pprint(gamma)


alpha =
1.35363
beta =
1.02689
gamma =
-0.543361
Rxyz =
⎡0.111496   -0.931193  -0.347058  0⎤
⎢                                  ⎥
⎢0.505324   -0.247589  0.826648   0⎥
⎢                                  ⎥
⎢-0.855696  -0.267545  0.442949   0⎥
⎢                                  ⎥
⎣    0          0          0      1⎦
alpha =
1.35363
beta =
1.11563
gamma =
-0.543361


In [30]:
cos(0.6)

0.825335614909678

In [27]:
acos(0.9999)

0.0141422534775121